### **Логика решения**

Наше решение построено на двухэтапном пайплайне с использованием двух различных больших языковых моделей (LLM) для достижения максимальной точности при решении математических задач.

#### **Этап 1: Первичное решение с помощью DeepMath-Zero-Math-7B**

1.  **Модель**: На первом этапе мы используем модель `zwhe99/DeepMath-Zero-Math-7B`. Эта модель является дообученной версией `Qwen/Qwen2.5-Math-7B` на специализированном датасете `DeepMath-103K`, который содержит сложные математические задачи. Размер модели составляет 7 миллиардов параметров, что соответствует требованиям конкурса (<9B).
2.  **Формирование промпта**: Для каждой задачи из тестового набора данных формируется стандартизированный промпт, который инструктирует модель рассуждать пошагово и заключать окончательный ответ в `\boxed{}`.
3.  **Генерация нескольких вариантов**: Чтобы повысить надежность, для каждой задачи мы генерируем 7 независимых решений (`n=7`). Это позволяет нам учесть вариативность ответов модели и выбрать наиболее вероятный правильный ответ.
4.  **Извлечение и выбор ответа**:
    *   Из каждого из 7 сгенерированных решений мы извлекаем содержимое, обернутое в `\boxed{}`.
    *   Среди всех извлеченных ответов мы выбираем наиболее часто встречающийся (моду). Этот подход, известный как "self-consistency", значительно повышает итоговую точность.
5.  **Форматирование**: Полученный ответ форматируется в соответствии с требованиями конкурса (например, замена `and` на `;`, обработка матриц и векторов) и сохраняется вместе с одним из полных решений, которое привело к этому ответу.

#### **Этап 2: Проверка и исправление с помощью Qwen3-8B (LLM-as-judge)**

1.  **Модель**: На втором этапе используется модель `Qwen/Qwen3-8B` в качестве "судьи". Эта модель имеет 8.2 миллиарда параметров, что также удовлетворяет условиям конкурса. Она обладает сильными логическими и математическими способностями.
2.  **Формирование промпта**: Модели-судье подается специальный промпт, который включает:
    *   Исходное условие задачи.
    *   Решение, сгенерированное моделью DeepMath на первом этапе.
    *   Инструкцию проверить решение, исправить возможные ошибки и предоставить окончательный корректный ответ в формате `\boxed{}`.
3.  **Повторная генерация и выбор ответа**: Аналогично первому этапу, мы генерируем 7 вариантов проверки и исправления от модели-судьи. Затем, используя тот же метод выбора наиболее частого ответа (мода), мы получаем финальный ответ.
4.  **Финальное форматирование**: Ответ, полученный от модели-судьи, проходит финальное форматирование для полного соответствия требованиям к сабмиту.

Этот двухэтапный подход позволяет сначала получить базовое решение от специализированной математической модели, а затем использовать более общую и мощную модель для проверки и исправления ошибок, что в совокупности повышает общую точность и надежность системы.

In [1]:
# Установка необходимых библиотек
# vllm - для быстрой и эффективной работы с моделями
# pylatexenc - для обработки LaTeX-форматирования в ответах
!pip install -U vllm==0.10.0
!pip install pylatexenc -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 

# 1. First Step (DeepMath-Zero-Math-7B)

In [2]:
%%writefile first_step.py
# Записываем весь пайплайн первого этапа в файл first_step.py

def main():
    # Импорт необходимых библиотек
    import os, math, numpy as np
    # Указываем, какие GPU использовать (в данном случае оба)
    os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
    # Устанавливаем seed для воспроизводимости результатов
    SEED = 777
    
    import random
    import numpy as np
    import os
    import torch
    
    # Функция для фиксации всех источников случайности
    def set_seed(seed: int):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        
        os.environ['PYTHONHASHSEED'] = str(seed)
    
    set_seed(SEED)
    
    import vllm
    import torch
    
    # Инициализация LLM с использованием vLLM
    llm = vllm.LLM(
        "zwhe99/DeepMath-Zero-Math-7B", # Название модели на Hugging Face
        tensor_parallel_size=2, # Распараллеливание модели на 2 GPU
        gpu_memory_utilization=0.95, # Использование 95% памяти GPU
        trust_remote_code=True, # Разрешение выполнения кода из хаба модели
        dtype="half", # Использование 16-битной точности для ускорения
        enforce_eager=True, # Включение eager-режима
        max_model_len=4096, # Максимальная длина последовательности
    )
    
    import pandas as pd
    # Загрузка тестовых данных
    test_df = pd.read_csv('/kaggle/input/aiijc-math/final_submission.csv')
    sub = pd.read_csv('/kaggle/input/aiijc-math/final_submission.csv')
    
    # Системный промпт, который будет использоваться для всех задач
    prompt = '''Please reason step by step, and put your final answer within \\boxed{}.'''
    
    # Вспомогательная функция для добавления контекста к задачам про партии
    def get_info(task):
        if 'парт' in task:
            return ". Размер партии - 100."
        return ''
    
    
    # Формирование датасета для модели в формате "сообщений"
    dataset = [
        [{"role": "system", "content": prompt},
        {"role": "user", "content": task + get_info(task)}] for task in test_df['task'].values]
    
    # Получение токенизатора из модели
    tokenizer = llm.get_tokenizer()
    # Функция для применения шаблона чата к датасету
    def formatting(dataset):
        texts = []
        for i in range(len(dataset)):
            texts.append(tokenizer.apply_chat_template(dataset[i], tokenize=False, add_generation_prompt=False))
        return texts
    
    # Применение форматирования ко всем задачам
    all_prompts = formatting(dataset)
    
    # Количество генерируемых ответов для каждой задачи (для self-consistency)
    N = 7
    
    # Генерация ответов моделью
    responses = llm.generate(
        all_prompts,
        vllm.SamplingParams(
            n=N,  # Количество вариантов ответа
            top_p=0.95,  # Семплирование из токенов, составляющих 95% вероятностной массы
            temperature=0.6,  # Уровень случайности генерации
            seed=SEED, # Seed для воспроизводимости
            skip_special_tokens=True,  # Пропускать специальные токены в выводе
            max_tokens=4096, # Максимальное количество токенов в ответе
        ),
        use_tqdm = True # Отображение прогресс-бара
    )
    
    import regex
    
    # Функция для извлечения содержимого из \boxed{}
    def extract_boxed_content(text):
        try:
            pattern = r'\\boxed({(?:[^{}]|(?1))*})'  # Рекурсивный паттерн для вложенных скобок
            matches = regex.findall(pattern, text)
            return [match[1:-1] for match in matches][-1]  # Возвращаем последнее совпадение без внешних скобок
        except:
            return ''
    
    # Функция для выбора наиболее частого ответа из N сгенерированных
    def get_answer(response):
        answers = []
        for i in range(N):
            ans = extract_boxed_content(response.outputs[i].text)
            if ans != '':
                answers.append(ans)
        if len(answers) == 0:
            answers.append('')
        return pd.Series(answers).mode()[0] # Возвращаем моду (самый частый ответ)
    
    # Применяем извлечение ответов ко всем сгенерированным результатам
    preds = [get_answer(responses[i]) for i in range(len(responses))]
    
    # Функция для форматирования ответа в соответствии с требованиями конкурса
    def format_answer(task, ans):
        if 'and' in ans:
            return ans.replace(' \\text{ and } ', '; ')
        if 'pmatrix' in ans:
            return LatexNodes2Text().latex_to_text(ans)[1:-1].strip()
        if ';' in task or 'matr' in task or 'матр' in task:
            return LatexNodes2Text().latex_to_text(ans).replace(',', ';')
        if 'frac' in ans:
            return LatexNodes2Text().latex_to_text(ans).replace(' \\text{ and } ', '; ')
        else:
            return ans.replace(' \\text{ and } ', '; ')
    
    from pylatexenc.latex2text import LatexNodes2Text
    
    # Применяем форматирование и записываем ответы в датафрейм
    sub['answer'] = [f'[{format_answer(task, x)}]' for task, x in zip(sub['task'], preds)]
    
    # Находим полные решения, которые соответствуют выбранному ответу
    solutions = []
    for i in range(len(sub)):
        for k in range(N):
            ans = format_answer(sub['task'].iloc[i], extract_boxed_content(responses[i].outputs[k].text))
            if f"[{ans}]" == sub['answer'].iloc[i]:
                solutions.append(responses[i].outputs[k].text)
                break
    
    # Дополнительная постобработка для векторов и других специфичных случаев
    sub.loc[['век' in x for x in sub['task']], 'answer'] = sub.loc[['век' in x for x in sub['task']], 'answer'].str.replace('(', '').str.replace(')', '').str.replace(',', ';')
    sub.loc[['(' not in x for x in sub['answer']], 'answer'] = sub.loc[['(' not in x for x in sub['answer']], 'answer'].str.replace(',', ';')
    
    # Сохраняем промежуточный сабмишен
    sub.to_csv('sub_deepmath-7b_n7.csv', index=False)
    # Добавляем столбец с полным решением и сохраняем для второго этапа
    sub['llm_solution'] = solutions
    sub.to_csv('deepmath-7b_solutions.csv', index=False)

if __name__ == '__main__':
    main()

Writing first_step.py


In [3]:
# Запускаем скрипт, написанный в предыдущей ячейке
!python first_step.py

2025-09-20 21:33:18.799885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758403999.060009     110 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758403999.135889     110 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO 09-20 21:33:36 [__init__.py:235] Automatically detected platform cuda.
config.json: 100%|█████████████████████████████| 796/796 [00:00<00:00, 4.79MB/s]
WARNING 09-20 21:33:54 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 09-20 21:33:54 [config.py:1604] Using max model len 4096
WARNING 09-20 21:33:54 [arg_utils.py:1690] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 09-20

# 2. Second Step (Qwen3-8B) | LLM-as-judge

In [4]:
# Импортируем базовые библиотеки и настраиваем окружение для второго этапа
import os, math, numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
SEED = 777

In [5]:
# Повторно определяем и вызываем функцию для фиксации seed
import random
import numpy as np
import os
import torch

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(SEED)

In [6]:
# Инициализируем вторую модель (судью)
import vllm
import torch

llm = vllm.LLM(
    "Qwen/Qwen3-8B", # Название модели-судьи
    tensor_parallel_size=2, 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=16000, # Увеличиваем максимальную длину для обработки задачи и решения
)

2025-09-20 22:17:28.634178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758406648.668402      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758406648.678002      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 09-20 22:17:37 [__init__.py:235] Automatically detected platform cuda.


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

WARNING 09-20 22:17:56 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 09-20 22:17:56 [config.py:1604] Using max model len 16000
WARNING 09-20 22:17:56 [arg_utils.py:1690] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 09-20 22:17:56 [cuda.py:103] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 09-20 22:17:56 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='Qwen/Qwen3-8B', speculative_config=None, tokenizer='Qwen/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=16000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, dec

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

INFO 09-20 22:20:37 [weight_utils.py:312] Time spent downloading weights for Qwen/Qwen3-8B: 113.856550 seconds


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


(VllmWorkerProcess pid=449) INFO 09-20 22:20:38 [weight_utils.py:312] Time spent downloading weights for Qwen/Qwen3-8B: 0.808139 seconds
INFO 09-20 22:21:43 [default_loader.py:262] Loading weights took 65.91 seconds
(VllmWorkerProcess pid=449) INFO 09-20 22:21:43 [default_loader.py:262] Loading weights took 65.05 seconds
(VllmWorkerProcess pid=449) INFO 09-20 22:21:44 [model_runner.py:1115] Model loading took 7.6394 GiB and 180.464383 seconds
INFO 09-20 22:21:44 [model_runner.py:1115] Model loading took 7.6394 GiB and 180.375324 seconds
(VllmWorkerProcess pid=449) INFO 09-20 22:21:55 [worker.py:295] Memory profiling takes 10.57 seconds
(VllmWorkerProcess pid=449) INFO 09-20 22:21:55 [worker.py:295] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
(VllmWorkerProcess pid=449) INFO 09-20 22:21:55 [worker.py:295] model weights take 7.64GiB; non_torch_memory takes 0.12GiB; PyTorch activation peak memory takes 1.30GiB; the rest of the m

In [7]:
# Загружаем данные, полученные на первом этапе
import pandas as pd
test_df = pd.read_csv('deepmath-7b_solutions.csv')
# Загружаем шаблон для финального сабмита
sub = pd.read_csv('/kaggle/input/aiijc-math/final_submission.csv')

In [8]:
# Промпт для модели-судьи
user_prompt = "You have been given a student's solution. Check it out step by step. Fix mistakes if there are any. All answers must be given. Put all the answers in one \\boxed{} separating the ';'. Put the final correct answer/answers in \\boxed{}"

In [9]:
# Та же вспомогательная функция для добавления информации о партиях
def get_info(task):
    if 'парт' in task:
        return ". Размер партии - 100."
    return ''

In [10]:
# Формируем датасет для модели-судьи, включая задачу и решение с первого этапа
dataset = [
    [{"role": "system", "content": user_prompt},
    {"role": "user", "content": "Task:\n" + task + get_info(task) + "\nStudent Solution:\n" + sol}] for task, sol in test_df[['task', 'llm_solution']].values]

In [11]:
# Функция для форматирования промптов с использованием токенизатора модели-судьи
tokenizer = llm.get_tokenizer()
def formatting(dataset):
    texts = []
    for i in range(len(dataset)):
        texts.append(tokenizer.apply_chat_template(dataset[i], tokenize=False, add_generation_prompt=False))
    return texts

In [12]:
# Применяем форматирование
all_prompts = formatting(dataset)

In [13]:
# Устанавливаем количество генерируемых ответов
N = 7

# Запускаем генерацию исправленных решений
responses = llm.generate(
    all_prompts,
    vllm.SamplingParams(
        n=N,  # Количество вариантов ответа
        top_p=0.95,  # Параметр семплирования
        temperature=0.6,  # Уровень случайности
        seed=SEED, # Seed для воспроизводимости
        skip_special_tokens=True,  # Пропускать спец. токены
        max_tokens=16000, # Максимальная длина ответа
    ),
    use_tqdm = True
)

Adding requests:   0%|          | 0/344 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2408 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

WARNING 09-20 22:22:48 [scheduler.py:1822] Sequence group 7_parallel_sample_6 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1
WARNING 09-20 22:29:57 [scheduler.py:1822] Sequence group 25_parallel_sample_3 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51
WARNING 09-20 22:36:30 [scheduler.py:1822] Sequence group 47_parallel_sample_3 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preem

In [14]:
# Используем те же функции для извлечения и выбора ответа
import regex

def extract_boxed_content(text):
    try:
        pattern = r'\\boxed({(?:[^{}]|(?1))*})'  # Рекурсивный паттерн
        matches = regex.findall(pattern, text)
        return [match[1:-1] for match in matches][-1]  # Удаляем внешние скобки
    except:
        return ''

def get_answer(response):
    answers = []
    for i in range(N):
        ans = extract_boxed_content(response.outputs[i].text)
        if ans != '':
            answers.append(ans)
    if len(answers) == 0:
        answers.append('')
    return pd.Series(answers).mode()[0] # Возвращаем моду


In [15]:
# Извлекаем финальные предсказания
preds = [get_answer(responses[i]) for i in range(len(responses))]

In [16]:
# Используем ту же функцию для финального форматирования ответов
def format_answer(task, ans):
    if 'and' in ans:
        return ans.replace(' \\text{ and } ', '; ')
    if 'pmatrix' in ans:
        return LatexNodes2Text().latex_to_text(ans)[1:-1].strip()
    if ';' in task or 'matr' in task or 'матр' in task:
        return LatexNodes2Text().latex_to_text(ans).replace(',', ';')
    if 'frac' in ans:
        return LatexNodes2Text().latex_to_text(ans).replace(' \\text{ and } ', '; ')
    else:
        return ans.replace(' \\text{ and } ', '; ')

In [17]:
# Убедимся, что библиотека установлена, и импортируем класс
!pip install pylatexenc -qq
from pylatexenc.latex2text import LatexNodes2Text

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
# Заполняем колонку 'answer' в финальном сабмишене
sub['answer'] = [f'[{format_answer(task, x)}]' for task, x in zip(sub['task'], preds)]

In [19]:
# Применяем финальную постобработку для векторов и других случаев
sub.loc[['век' in x for x in sub['task']], 'answer'] = sub.loc[['век' in x for x in sub['task']], 'answer'].str.replace('(', '').str.replace(')', '').str.replace(',', ';')
sub.loc[['(' not in x for x in sub['answer']], 'answer'] = sub.loc[['(' not in x for x in sub['answer']], 'answer'].str.replace(',', ';')

In [20]:
# Сохраняем итоговый файл решения
sub.to_csv('sub_deepmath-7b_n7__qwen3-8b-as-judge-n7.csv', index=False)

In [21]:
# Выводим пример для проверки: первая задача и первое сгенерированное решение
i = 0
k = 0
print("Task:\n" + test_df['task'].iloc[i])
print("Solution:\n" + responses[i].outputs[k].text)

Task:
Вычислить $\displaystyle \lim_{n \to \infty} \frac{n \cos(n^2 + e^n)}{2^n - 1}$. Ответ запишите в виде десятичной дроби, округлив по правилам математического округления до 3 знаков после запятой.
Solution:


To evaluate the limit:

$$
\lim_{n \to \infty} \frac{n \cos(n^2 + e^n)}{2^n - 1},
$$

we analyze the behavior of the numerator and the denominator as $n \to \infty$.

---

### Step 1: Behavior of the Numerator

The numerator is:

$$
n \cos(n^2 + e^n)
$$

The cosine function, $\cos(x)$, is bounded between $-1$ and $1$ for all real $x$. Therefore:

$$
|n \cos(n^2 + e^n)| \leq n
$$

This means the numerator is bounded in absolute value by $n$.

---

### Step 2: Behavior of the Denominator

The denominator is:

$$
2^n - 1
$$

As $n \to \infty$, the term $2^n$ grows exponentially, and $-1$ becomes negligible. So:

$$
2^n - 1 \approx 2^n
$$

Therefore, the denominator grows exponentially.

---

### Step 3: Bounding the Expression

We can now bound the absolute value of the full exp